In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim

In [47]:
import torch
import torch.utils.data as data
import os
import os.path
#from plyfile import PlyData, PlyElement
from plyfile import PlyData
import numpy as np

def load_ply(file_name, with_faces=False, with_color=False):
    
    ply_data = PlyData.read(file_name)
    points = ply_data['vertex']
    points = np.vstack([points['x'], points['y'], points['z']]).T
    
    return points

def load_list(root, train = 'train'):
    input_dir = []
    rootdir = root
    #rootdir = '/home/cdi0/data/shape_net_core_uniform_samples_2048_split/'

    if train =='train':
        rootdir = os.path.join(rootdir, train)

        for dirs in os.listdir(rootdir):
            if dirs == 'train_0':
                target_dir = os.path.join(rootdir, dirs)
            elif dirs.startswith('train'):
                input_dir.append(os.path.join(rootdir, dirs))

    else:
        rootdir = os.path.join(rootdir, 'test') 

        for dirs in os.listdir(rootdir):
            if dirs == 'test_0':
                target_dir = os.path.join(rootdir, dirs)
            elif dirs.startswith('test'):
                input_dir.append(os.path.join(rootdir, dirs))

    input_dir.sort()

    input_data_list = []
    target_data_list = []

    
    for i in input_dir:
        lst = []
        for dirpath, dirnames, filenames in os.walk(i):
            for filename in [f for f in filenames if f.endswith(".ply")]:
                lst.append(os.path.join(dirpath, filename))
        lst.sort()
        input_data_list.append(lst)

    for dirpath, dirnames, filenames in os.walk(target_dir):
            for filename in [f for f in filenames if f.endswith(".ply")]:
                target_data_list.append(os.path.join(dirpath, filename))

    target_data_list.sort()

    input_set_list = []
    for i in range(len(input_data_list)):
        lst = []
        for j in range(len(input_data_list[i])):
            lst.append((input_data_list[i][j], target_data_list[j]))
        input_set_list.append(lst)
        
    return input_set_list

class ShapeNetDataset(data.Dataset):
    def __init__(self, dir, train = 'train', n_points = 2048, augmentation = False, stage = 0, opt = None):
        
        self.root = dir
        self.loader = load_ply
        self.opt = opt
        self.train = train
        
        lst = []
        l = load_list(dir, self.train)
        for i in range(stage+1):
            lst = lst + l[i]
            
        self.lst = lst
        self.loader = load_ply
        
    def __getitem__(self, idx):
    
        input_pcd, target_pcd = self.lst[idx]
        input_pcd = self.loader(input_pcd)
        target_pcd = self.loader(target_pcd)
        
        mask = np.isin(target_pcd, input_pcd)
        m = np.all(mask, axis = 1)
        
        t = np.zeros((target_pcd.shape[0],4))
        t[:,3] = m

        n = 0
        for i in range(len(m)):
            if m[i] == 1:
                t[i,:3] = input_pcd[n]
                n +=1
            else:
                t[i,:3] = np.random.randn(1,3) / 3
                
        input_pcd = t
        
        return input_pcd, target_pcd, m
    
    
    def __len__(self):
        return len(self.lst)
    

In [48]:
class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(4, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 12)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        #iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        iden = Variable(torch.cat((torch.eye(3).repeat(batchsize,1,1), torch.zeros(batchsize, 1, 3)), dim = 1))
        if x.is_cuda:
            iden = iden.cuda()
        x = x.view(-1, 4, 3)
        x = x + iden
        
        return x

In [49]:
class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = False):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat

In [50]:
class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = False):
        super(PointNetfeat, self).__init__()
        self.stn = STN3d()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1)
        
class PointNetCls(nn.Module):
    def __init__(self, k=2, feature_transform=False):
        super(PointNetCls, self).__init__()
        self.feature_transform = feature_transform
        self.feat = PointNetfeat(global_feat=True, feature_transform=feature_transform)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.dropout = nn.Dropout(p=0.3)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1), trans, trans_feat


class PointNetDenseCls(nn.Module):
    def __init__(self, feature_transform=False):
        super(PointNetDenseCls, self).__init__()
        #self.k = k
        self.feature_transform=feature_transform
        self.feat = PointNetfeat(global_feat=False, feature_transform=feature_transform)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, 3, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x = self.feat(x)
        print(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        #x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, 3)
        return x


In [6]:
pcls = PointNetDenseCls()
pcls.to(device = 'cuda:0')

PointNetDenseCls(
  (feat): PointNetfeat(
    (stn): STN3d(
      (conv1): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=12, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
 

In [51]:
device = 'cuda:0'

In [52]:
rootdir = '/home/cdi0/data/shape_net_core_uniform_samples_2048_split/'

In [53]:
dataset = ShapeNetDataset(
    dir=rootdir,
    )
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=int(4))

test_dataset = ShapeNetDataset(
    dir=rootdir,
    train='test',
    )
testdataloader = torch.utils.data.DataLoader(
    test_dataset,    batch_size=32,
    shuffle=True,
    num_workers=int(4))

print(len(dataset), len(test_dataset))

48803 8646


In [55]:
points, target, mask = iter(testdataloader).next()
print(target.shape)
points = points.transpose(2, 1).contiguous()
points = points.to(device='cuda:0', dtype=torch.float)
target = target.to(device='cuda:0', dtype=torch.float)

torch.Size([32, 2048, 3])


In [57]:
target.shape

torch.Size([32, 2048, 3])

In [11]:
from pointnet.model import PointNetDenseCls, feature_transform_regularizer
classifier = PointNetDenseCls()
classifier.to(device = 'cuda:1')
optimizer = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

optimizer.zero_grad()

pred = classifier(points)

In [12]:
target[0].shape

torch.Size([2048, 3])

In [34]:
def pairwise_dist(x, y):
    xx, yy, zz = torch.mm(x, x.t()), torch.mm(y, y.t()), torch.mm(x, y.t())
    rx = xx.diag().unsqueeze(0).expand_as(xx)
    ry = yy.diag().unsqueeze(0).expand_as(yy)
    P = rx.t() + ry - 2 * zz
    return P


def NN_loss(x, y, dim=0):
    dist = pairwise_dist(x, y)
    values, indices = dist.min(dim=dim)
    return values.mean()


def distChamfer(a, b):
    x, y = a, b
    bs, num_points, points_dim = x.size()
    xx = torch.bmm(x, x.transpose(2, 1))
    yy = torch.bmm(y, y.transpose(2, 1))
    zz = torch.bmm(x, y.transpose(2, 1))
    diag_ind = torch.arange(0, num_points).type(torch.cuda.LongTensor)
    rx = xx[:, diag_ind, diag_ind].unsqueeze(1).expand_as(xx)
    ry = yy[:, diag_ind, diag_ind].unsqueeze(1).expand_as(yy)
    P = rx.transpose(2, 1) + ry - 2 * zz
    return P.min(1)[0], P.min(2)[0]
    #return torch.min(P, 1)[0], torch.min(P, 2)[0], torch.min(P, 1)[1], torch.min(P, 2)[1]


In [35]:
dist1, dist2 = distChamfer(target, pred)

In [40]:
 loss = (torch.mean(dist1)) + (torch.mean(dist2))

In [59]:
mask_ = mask.unsqueeze(2).repeat(1,1,3)
mask__ = ~mask_
mask__ = mask__.to(device, dtype = torch.float32)
mask_ = mask_.to(device, dtype = torch.float32)

pred = (pred * mask__) + (target * mask_)

In [70]:
mask.unsqueeze(2).repeat(1,1,3).shape

torch.Size([32, 2048, 3])

In [61]:
mask__.shape

torch.Size([32, 2048, 3])

In [63]:
target.shape

torch.Size([32, 2048, 3])

In [65]:
target_mask = target * mask__

In [73]:
target_mask[target_mask.sum(dim = 2) != 0].view(32,-1,3).shape

torch.Size([32, 410, 3])

In [ ]:
target_mask[target_mask]

In [48]:
from emd import EMDLoss

mask_ = mask.unsqueeze(2).repeat(1,1,3)
mask__ = ~mask_

dist =  EMDLoss()

a = pred[mask__].view(32,-1,3)
b = target[mask__].view(32,-1,3)




In [49]:
cost = dist(a, b)
loss = torch.sum(cost)

loss.backward()


In [57]:
loss.item()

4744.01708984375

In [2]:
label_real = torch.full((32,), 1)
label_fake = torch.full((32,), 0)

In [4]:
label = torch.stack((label_real, label_fake), dim = 1)

In [14]:
 torch.stack((torch.ones((6)), torch.zeros(6)), dim = 1).shape

torch.Size([6, 2])

In [27]:
criterion = nn.MSELoss()

cost = criterion(pred, target)
cost.backward()

In [31]:
optimizer.step()

In [30]:
cost

tensor(0.0511, device='cuda:0', grad_fn=<MseLossBackward>)

In [58]:
a = np.array([1,2,3])

In [75]:
import open3d as o3d     # Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.io.write_point_cloud("test.ply", pcd)

RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)

In [80]:
from pyntcloud import PyntCloud
import pandas as pd

In [92]:
target = target.cpu().numpy()

In [94]:
cloud = PyntCloud(pd.DataFrame(
    # same arguments that you are passing to visualize_pcl
    data=target[0].reshape(2048,-1),
    columns=["x", "y", "z"]))
cloud.to_file("output.ply")

In [24]:
a = torch.tensor([[1,2,3,0],[1,0,3,4],[0,0,0,0],[1,2,3,4,]])

In [42]:
a.size(0)

4

In [23]:
a[a.nonzero()[0,1]]

tensor([1, 2, 3, 0])

In [28]:
a.where(~a.any(axis = 1))

RuntimeError: all only supports torch.uint8 and torch.bool dtypes

In [39]:
a[a.sum(dim =1)!=0]

tensor([[1, 2, 3, 0],
        [1, 0, 3, 4],
        [1, 2, 3, 4]])

In [45]:
torch.full((3,), 1)

tensor([1., 1., 1.])